# Слияние таблиц

Давайте представим, что мы аналитики в компании, которая производит и продает скрепки. Нам нужно составить несколько отчетов для отдела продаж, чтобы посмотреть, как у них дела. Данные лежат в следующих словарях:

In [1]:
import numpy as np
import pandas as pd

In [2]:
# кол-во скрепок, проданных сотрудниками
sales = {'Tony': 103,
         'Sally': 202,
         'Randy': 380,
         'Ellen': 101,
         'Fred': 82
        }

# все сотрудники и в каком регионе они работают
region = {'Tony': 'West',
          'Sally': 'South',
          'Carl': 'West',
          'Archie': 'North',
          'Randy': 'East',
          'Ellen': 'South',
          'Fred': np.nan,
          'Mo': 'East',
          'HanWei': np.nan,
         }

Мы можем создать два отдельных dataframe из словарей 

In [3]:
sales_df = pd.DataFrame.from_dict(sales, 
                                  orient='index', 
                                  columns=['sales'])

region_df = pd.DataFrame.from_dict(region, 
                                   orient='index', 
                                   columns=['region'])

In [4]:
sales_df

,sales
Tony,103
Sally,202
Randy,380
Ellen,101
Fred,82


In [5]:
region_df

,region
Tony,West
Sally,South
Carl,West
Archie,North
Randy,East
Ellen,South
Fred,NaN
Mo,East
HanWei,NaN


Теперь давайте объединим все наши данные в один датафрейм.

### Join
Давайте начнем с join, потому что он самый простой. У датафреймов данных есть параметр index. Это ключ вашей таблицы, и если мы знаем индекс, то мы можем легко получить строку, содержащую наши данные, используя .loc. Если вы напечатаете свой датафрейм, вы увидите индекс в крайнем левом столбце. Еще его можно получить, напрямую использовав .index:

In [6]:
sales_df.index

Index(['Tony', 'Sally', 'Randy', 'Ellen', 'Fred'], dtype='object')

Значит индекс в sales_df - это имя продавца. Учитывая индекс, мы можем найти данные строки так:

In [7]:
sales_df.loc['Tony']

sales    103
Name: Tony, dtype: int64

Вернемся к join'ам. Метод join берет два датафрейма и соединяет их по индексам (технически вы можете выбрать столбец для объединения для левого датафрейма). Давайте посмотрим, что происходит, если мы объединим(aka сджойним) наши два датафрейма с помощью метода join:

In [8]:
joined_df = region_df.join(sales_df, how='left')
joined_df

,region,sales
Tony,West,103.0
Sally,South,202.0
Carl,West,NaN
Archie,North,NaN
Randy,East,380.0
Ellen,South,101.0
Fred,NaN,82.0
Mo,East,NaN
HanWei,NaN,NaN


Метод join использует индекс или указанный столбец из левого датафрейма в качестве ключа джойна. Таким образом, столбец, по которому мы джойним левый датафрейм, не обязательно должен быть его индексом. Но вот для правого датафрейма ключ джойна должен быть его индексом ОБЯЗАТЕЛЬНО. 

Можно воспринимать метод join как объединение на основе индекса и использовать merge, если не хочется привязываться к индексам.

В объединенном датафрейме есть несколько NaN. Так произошло , потому что не у всех сотрудников были продажи. Те, у кого не было продаж, отсутствуют в sales_df, но мы по-прежнему отображаем их, потому что мы выполнили left join (указав «how = left»), которое возвращает все строки из левого датафрейма region_df, независимо от того, есть ли совпадение в правом. Если мы не хотим отображать какие-либо NaN в нашем результате соединения, мы вместо этого можем сделать inner join (указав how = "inner").

<img src="https://files.realpython.com/media/join_diagram.93e6ef63afbe.png">

In [9]:
joined_df = region_df.join(sales_df, how='inner')
joined_df

,region,sales
Tony,West,103
Sally,South,202
Randy,East,380
Ellen,South,101
Fred,NaN,82


### Merge

На базовом уровне Merge делает более или менее то же самое, что и join. Оба метода используются для объединения двух датафреймов, но merge является более универсальным за счет более подробного описания входных данных. Давайте посмотрим, как мы можем создать тот же объединенный датафрейм с помощью merge:

In [21]:
new_region_df = region_df.reset_index(names=['name'])
new_region_df

,name,region
0,Tony,West
1,Sally,South
2,Carl,West
3,Archie,North
4,Randy,East
5,Ellen,South
6,Fred,NaN
7,Mo,East
8,HanWei,NaN


In [22]:
new_sales_df = sales_df.reset_index(names=['firstname'])
new_sales_df

,firstname,sales
0,Tony,103
1,Sally,202
2,Randy,380
3,Ellen,101
4,Fred,82


In [26]:
joined_df_merge = new_region_df.merge(new_sales_df, how='left', 
                                left_on="name",
                                right_on='firstname')
joined_df_merge

,name,region,firstname,sales
0,Tony,West,Tony,103.0
1,Sally,South,Sally,202.0
2,Carl,West,NaN,NaN
3,Archie,North,NaN,NaN
4,Randy,East,Randy,380.0
5,Ellen,South,Ellen,101.0
6,Fred,NaN,Fred,82.0
7,Mo,East,NaN,NaN
8,HanWei,NaN,NaN,NaN


In [29]:
joined_df_merge2 = new_region_df.merge(new_sales_df.rename(columns={'firstname':'name'}), 
                                       how='left', 
                                       on="name")
joined_df_merge2

,name,region,sales
0,Tony,West,103.0
1,Sally,South,202.0
2,Carl,West,NaN
3,Archie,North,NaN
4,Randy,East,380.0
5,Ellen,South,101.0
6,Fred,NaN,82.0
7,Mo,East,NaN
8,HanWei,NaN,NaN


In [31]:
help(region_df.merge)

Help on method merge in module pandas.core.frame:

merge(right: 'DataFrame | Series', how: 'str' = 'inner', on: 'IndexLabel | None' = None, left_on: 'IndexLabel | None' = None, right_on: 'IndexLabel | None' = None, left_index: 'bool' = False, right_index: 'bool' = False, sort: 'bool' = False, suffixes: 'Suffixes' = ('_x', '_y'), copy: 'bool' = True, indicator: 'bool' = False, validate: 'str | None' = None) -> 'DataFrame' method of pandas.core.frame.DataFrame instance
    Merge DataFrame or named Series objects with a database-style join.
    
    A named Series object is treated as a DataFrame with a single named column.
    
    The join is done on columns or indexes. If joining columns on
    columns, the DataFrame indexes *will be ignored*. Otherwise if joining indexes
    on indexes or indexes on a column or columns, the index will be passed on.
    When performing a cross merge, no column specifications to merge on are
    allowed.
    
    .. warning::
    
        If both key co

Merge полезен, когда мы не хотим привязываться к индексам. Скажем, мы хотим знать, сколько, в процентном отношении,  каждый сотрудник внес в продажи в своем регионе. Мы можем использовать groupby для суммирования всех продаж в каждом уникальном регионе. В приведенном ниже коде reset_index используется, чтобы превратить регион из индекса в обычную колонку.

In [40]:
grouped_df = joined_df_merge2.groupby(by='region')['sales'].sum().reset_index()
grouped_df

,region,sales
0,East,380.0
1,North,0.0
2,South,303.0
3,West,103.0


Теперь остается смержить join_df_merge с grouped_df, используя столбец region. Мы должны указать суффикс, потому что оба наших блока данных (которые мы объединяем) содержат столбец с названием sales. Входные суффиксы добавляют указанные строки к меткам столбцов с одинаковыми именами в обоих датафреймах (это удобно, потому что ничего не перепутается). В нашем случае, поскольку столбец продаж второго датафрейма фактически отражает продажи во всем регионе, мы можем добавить суффикс «_region».

In [48]:
employee_contrib = joined_df_merge2.merge(grouped_df, how='left', 
                                         on='region',
                                         suffixes=('','_region'))
employee_contrib

,name,region,sales,sales_region
0,Tony,West,103.0,103.0
1,Sally,South,202.0,303.0
2,Carl,West,NaN,103.0
3,Archie,North,NaN,0.0
4,Randy,East,380.0,380.0
5,Ellen,South,101.0,303.0
6,Fred,NaN,82.0,NaN
7,Mo,East,NaN,380.0
8,HanWei,NaN,NaN,NaN


Теперь у нас есть исходный столбец продаж и новый столбец sales_region, в котором указывается общий объем продаж в регионе. Давайте посчитаем процент продаж каждого сотрудника, а затем очистим наш датафрейм, отбросив наблюдения, без региона (Fred и HanWei), и заполняя NaN в столбце продаж нулями.

In [52]:
employee_contrib = employee_contrib.dropna(subset=['region']) # уберем регионы, которые не указаны

employee_contrib = employee_contrib.fillna({'sales': 0}) # заполним пустые ячейки 0

employee_contrib['%_of_sales'] = round(employee_contrib['sales']/employee_contrib['sales_region'], 2)

employee_contrib[['name','region','sales','%_of_sales']].sort_values(by=['region','%_of_sales'])

,name,region,sales,%_of_sales
7,Mo,East,0.0,0.00
4,Randy,East,380.0,1.00
3,Archie,North,0.0,NaN
5,Ellen,South,101.0,0.33
1,Sally,South,202.0,0.67
2,Carl,West,0.0,0.00
0,Tony,West,103.0,1.00
